<a href="https://colab.research.google.com/github/vitroid/ClustIce/blob/tilecycles/clustice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install py3Dmol genice-core

In [ ]:
pip install git+https://github.com/vitroid/ClustIce.git

In [ ]:
pip install .

In [ ]:
pip uninstall clusitce

In [ ]:
from logging import getLogger, INFO, DEBUG, basicConfig
import numpy as np
import networkx as nx
import py3Dmol

from clustice.geometry import make_layout
from clustice.gromacs import render
from genice_core import ice_graph
from genice_core.serialize import serialize
from clustice.water import tip4p
from clustice import graph


logger = getLogger()
basicConfig(level=INFO)

# O-O distance
L = 0.27

# note: g must be a graph whose labels start from 0.
# g = graph.great_icosahedron(12, separation=L)
g = graph.great_decahedron(12)
# g = graph.small_barrelan()
# g = graph.large_barrelan()
# g = graph.twistane()
# g = graph.adamantane()
# g = nx.cycle_graph(6) # hexagon
# g = nx.cycle_graph(7) # heptagon
# g = nx.cubical_graph() # cubic octamer
# g = nx.dodecahedral_graph()


if "pos" in g.nodes[0]:
    # extract the embedded coords in g
    layout = np.array([g.nodes[v]["pos"] for v in g])
else:
    # estimate of the positions of the nodes
    layout = make_layout(g, edgelen=L)

# set orientations of the hydrogen bonds.
# if pos is given, the net dipole moment is minimized.
dg = ice_graph(g, vertexPositions=layout, dipoleOptimizationCycles=100)
# dg = ice_graph(g)

# get the unique id for the graph
# id = serialize(dg)
# print(id)

# put water molecules
gro = render(dg, layout, watermodel=tip4p)
with open("save.gro", "w") as f:
    f.write(gro)

# show
view = py3Dmol.view()
view.addModel(gro, "gro")
view.setStyle({"stick": {}})
view.addUnitCell()
view.zoomTo()
view.show()

Rebuild the cluster from id.

In [ ]:
import networkx as nx

from clustice.geometry import make_layout
from clustice.gromacs import render
from genice_core.serialize import deserialize
from clustice.water import tip4p

import py3Dmol

s = "1.2.3+6.19.3.4+6.7.14.1+7.8+13.0+9.10+12.16.12.13+15.5.15+17.4+18.11.0+18"

dg = deserialize(s)

# Make layout of the nodes
g = nx.Graph(dg)
L = 0.27
layout = make_layout(g, edgelen=L)

# put water molecules
gro = render(dg, layout, watermodel=tip4p)

# show
view = py3Dmol.view()
view.addModel(gro, "gro")
view.setStyle({"stick": {}})
view.addUnitCell()
view.zoomTo()
view.show()

In [ ]:
pip install GenIce2

In [ ]:
import networkx as nx
from genice2.genice import GenIce
from genice2.plugin import Format, Lattice

from clustice.gromacs import render
from genice_core import ice_graph
from clustice.water import tip4p

import py3Dmol

lattice = Lattice("1h")
formatter = Format("raw", stage=(1, 2))
raw = GenIce(lattice, signature="Ice Ih", rep=(3, 3, 3)).generate_ice(formatter)

# graph is the topology of the hydrogen-bond network
g = nx.Graph(raw["graph"])
# reppositions contains the positions of CoM of water in fractional coordinate
layout = raw["reppositions"]
# repcell is the cell matrix (transposed)
cell = raw["repcell"]

# set orientations of the hydrogen bonds.
# It automatically depolarizes by optimizing the arrangement.
dg = ice_graph(
    g, vertexPositions=layout, isPeriodicBoundary=True, dipoleOptimizationCycles=200
)

# put water molecules
gro = render(
    dg,
    layout @ cell,
    watermodel=tip4p,
    cell=f"{cell[0,0]} {cell[1,1]} {cell[2,2]}",
    pbc=True,
)

# show
view = py3Dmol.view()
view.addModel(gro, "gro")
view.setStyle({"stick": {}})
view.addUnitCell()
view.zoomTo()
view.show()

# Benchmark tests

In [ ]:
from clustice.geometry import make_layout
from clustice.graph import great_icosahedron

from logging import getLogger, INFO

logger = getLogger()
logger.setLevel(INFO)

# O-O distance
L = 0.27

# note: g must be a graph whose labels start from 0.
# g = nx.dodecahedral_graph() # dodecahedral 20mer
g = great_icosahedron(1)
layout = make_layout(g, edgelen=L)
print(len(layout))


In [ ]:
%%timeit

# estimate of the positions of the nodes
layout = make_layout(g, edgelen=L)



In [ ]:
%%timeit
dg = ice_graph(g, vertexPositions=layout)



In [ ]:
%%timeit
dg = ice_graph(g)


In [ ]:
# great_icosahedron
# size  nnode   constellation        depolarization       ice rule
benchmark=[
    [0, 20,  0.146, 0.00612, 0.000410],
    [1, 100, 0.676, 0.00865, 0.00229],
    [2, 280, 3.14,  0.0178,  0.00569],
    [3, 600, 51.4,  0.024,   0.0133]